In [1]:
import torch
from torch import nn
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from d2l import torch as d2l

In [2]:
def nin_block(in_chanels, out_chanels, kernel_size, strides, padding):
    return nn.Sequential(nn.Conv2d(in_chanels, out_chanels, kernel_size, strides,padding),
                         nn.ReLU(),
                        nn.Conv2d(out_chanels, out_chanels,kernel_size=1), nn.ReLU(),
                        nn.Conv2d(out_chanels, out_chanels, kernel_size=1),nn.ReLU())

In [3]:
net = nn.Sequential(nin_block(1, 96, kernel_size=11, strides=4, padding=0),
                   nn.MaxPool2d(3, stride=2),
                   nin_block(96, 256, kernel_size=5, strides=1, padding=2),
                   nn.MaxPool2d(3, stride=2),
                   nin_block(256, 384, kernel_size=3, strides=1, padding=1),
                   nn.MaxPool2d(3, stride=2),
                   nn.Dropout(0.5),
                   nin_block(384, 10, kernel_size=3, strides=1, padding=1),
                   nn.AdaptiveAvgPool2d((1, 1)),
                   nn.Flatten())

In [5]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [ ]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

In [13]:
!nvidia-smi

Wed Aug  9 03:01:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.69       Driver Version: 511.69       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:03:00.0 Off |                  N/A |
| N/A   47C    P8    N/A /  N/A |   1777MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------